In [1]:
# Test inference pipeline

In [2]:
from flair.data import Sentence
from flair.models import MultiTagger
from flair.models import TextClassifier

In [3]:
# load the 'chunk' and POS taggers
tagger = MultiTagger.load(['chunk-fast', 'pos-fast'])

2021-03-08 00:06:08,846 loading file C:\Users\e0185181\.flair\models\en-chunk-conll2000-fast-v0.4.pt
2021-03-08 00:06:09,214 loading file C:\Users\e0185181\.flair\models\en-pos-ontonotes-fast-v0.5.pt


In [95]:
# load sentiment tagger
classifier = TextClassifier.load('sentiment')

2021-03-08 01:37:01,945 loading file C:\Users\e0185181\.flair\models\sentiment-en-mix-distillbert_3.1.pt


In [153]:
# make a sentence
# text = 'I love Bamboo HR, but I really hate interviews.'
# text = 'I think she is really cool but maybe too cool'
text = 'I like the Samsung smart watch because it is sleek and durable'
sentence = Sentence(text)

In [154]:
# run NER over sentence
tagger.predict(sentence)

In [155]:
# check prediction
print(sentence)

Sentence: "I like the Samsung smart watch because it is sleek and durable"   [− Tokens: 12  − Token-Labels: "I <S-NP/PRP> like <S-VP/VBP> the <B-NP/DT> Samsung <I-NP/NNP> smart <I-NP/JJ> watch <E-NP/NN> because <S-SBAR/IN> it <S-NP/PRP> is <S-VP/VBZ> sleek <B-ADJP/JJ> and <I-ADJP/CC> durable <E-ADJP/JJ>"]


In [156]:
def get_conjunctions(sentence):
    pos = [span.tag for span in sentence.get_spans('pos-fast')]
    has_conjunction = 'CC' in pos
    breaks = []
    if has_conjunction:
        for i, val in enumerate(pos):
            if val == 'CC':
                breaks.append(i)
    return breaks

In [157]:
breaks = get_conjunctions(sentence)

In [158]:
print(pos)
print(has_conjunction)
print(breaks)

['PRP', 'VBP', 'NNP', 'NNP', ',', 'CC', 'PRP', 'RB', 'VBP', 'NNS', '.']
True
[10]


In [159]:
def combine_spans(spans):
    text = [text.to_original_text() for text in spans]
    return ' '.join(text)

In [160]:
parts = []
spans = sentence.get_spans('pos-fast')
current_break = 0

for next_cc in breaks:
    before_cc = spans[current_break:next_cc]
    cc = spans[next_cc]
    parts.append({'type': 'phrase', 'text': combine_spans(before_cc)})
    parts.append({'type': 'conjunction', 'text': cc.text})
    current_break = next_cc

last_part = spans[breaks[-1]+1:]
parts.append({'type': 'phrase', 'text': combine_spans(last_part)})

In [161]:
parts

[{'type': 'phrase',
  'text': 'I like the Samsung smart watch because it is sleek'},
 {'type': 'conjunction', 'text': 'and'},
 {'type': 'phrase', 'text': 'durable'}]

In [162]:
for part in parts:
    if part['type'] == 'phrase':
        sentence = Sentence(part['text'])
        classifier.predict(sentence)
        part['sentiment'] = sentence.to_dict()['labels'][0]['value']
        part['labels'] = sentence.to_dict()['labels'][0]['confidence']
    if part['type'] == 'conjunction':
        if part['text'] in ('but'):
            part['reverse'] = True
        elif part['text'] in ('and', 'or'):
            part['reverse'] = False
        else:
            part['reverse'] = None
            print('Error.. unknown conjunction')

In [165]:
parts

[{'type': 'phrase',
  'text': 'I like the Samsung smart watch because it is sleek',
  'sentiment': 'POSITIVE',
  'labels': 0.9975072741508484},
 {'type': 'conjunction', 'text': 'and', 'reverse': False},
 {'type': 'phrase',
  'text': 'durable',
  'sentiment': 'POSITIVE',
  'labels': 0.999009370803833}]

In [171]:
import termcolor

In [174]:
hello=termcolor.colored('Hello World!', color='green', on_color='on_grey')
print(hello)

Hello World!


In [176]:
from colorama import Fore
print(Fore.RED + 'Hello')

Hello
